In [1]:
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

/bin/bash: /Users/jonathankernes/Documents/Computer: No such file or directory
Note: you may need to restart the kernel to use updated packages.


In [2]:
URL='https://pitchfork.com/reviews/albums/juice-wrld-legends-never-die/'
page=requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [61]:
# Input a review page URL, returns a tuple of text score
# this is specific to the pitchfork review page. Could change over time!
def get_text_and_score(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    pElems = soup.find('div', class_='contents dropcap')
    text = ''.join([p.get_text() for p in pElems.find_all('p')])
    score = float(soup.find('span', class_='score').get_text())
    return text, score

In [62]:
def extract_all_reviews(num_pages=10):
    # Initialize the album names, text column, and score colum
    album_names = []
    review_text = []
    review_score = []
    start=time.time()
    
    #Loop over all pages. Each page has about 16 albums on it
    for page_num in range(1,num_pages+1):
        print('Scraping page',page_num,'...',time.time()-start,'s elapsed')
        
        # Extract the URL of the main page, and dump its contents into page/soup
        URL = 'https://pitchfork.com/reviews/albums/?genre=rap&page='+str(page_num)
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        # Now loop over all linked albums on main page
        # apply the text, score extraction to each review
        reviews = soup.find_all('div', class_='review')
        for review in reviews:
            title = review.find('h2',class_='review__title-album').get_text()
            print('\tGetting review/score for',title)
            review_link = 'https://pitchfork.com/'+review.find('a')['href']
            text, score = get_text_and_score(review_link)
            album_names.append(title)
            review_text.append(text)
            review_score.append(score)
            
    # Lastly, create a dataframe to store the data, and output it
    print('Total time to scrape:',time.time()-start)
    df = pd.DataFrame({'Score':review_score,'Review':review_text}, index = album_names)
    return df

In [63]:
df = extract_all_reviews(10)

Scraping page 1 ... 4.0531158447265625e-06 s elapsed
	Getting review/score for Legends Never Die
	Getting review/score for None of Us Are Getting Out of This Life Alive
	Getting review/score for Barnacles
	Getting review/score for One Year
	Getting review/score for As God Intended
	Getting review/score for So Icy Summer
	Getting review/score for Shoot for the Stars Aim for the Moon
	Getting review/score for Futuro Conjunto
	Getting review/score for City on Lock
	Getting review/score for Everything’s Strange Here
	Getting review/score for Weight of the World
	Getting review/score for Young & Turnt 2 (Deluxe)
Scraping page 2 ... 31.162528038024902 s elapsed
	Getting review/score for Son of a Gun
	Getting review/score for Roy Ayers JID 002
	Getting review/score for Little Dominiques Nosebleed
	Getting review/score for I Love You Say It Back
	Getting review/score for The Godfather 3
	Getting review/score for Thank You For Using GTL
	Getting review/score for Shrines
	Getting review/score fo

In [64]:
df.to_csv('AlbumReviewData')

In [65]:
df.head()

,Score,Review
Legends Never Die,7.1,"At 21, Juice WRLD had begun to master the worl..."
None of Us Are Getting Out of This Life Alive,5.5,"In 2011, after nearly a decade of attempted re..."
Barnacles,7.3,"Of Atlanta’s abundance of rap weirdos, the 23-..."
One Year,7.8,Model Home’s music would blend right in with a...
As God Intended,7.7,"Halfway through As God Intended, on the ominou..."


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120 entries, Legends Never Die to The Hood Dictionary
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Score   120 non-null    float64
 1   Review  120 non-null    object 
dtypes: float64(1), object(1)
memory usage: 2.8+ KB


In [67]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X=vectorizer.fit_transform(df['Review'])
y = df['Score']

In [76]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [77]:
from sklearn.ensemble import RandomForestRegressor

forest_clf = RandomForestRegressor()
forest_clf.fit(X_train, y_train)
y_pred = forest_clf.predict(X_test)

In [78]:
y_pred

array([5.586, 7.322, 7.06 , 7.393, 7.023, 7.205, 7.192, 7.004, 6.927,
       7.083, 7.074, 7.191])

In [79]:
y_test

38 Baby 2              7.0
BLQLYTE                7.5
As God Intended        7.7
Self Hate Wraith       7.2
High Off Life          7.1
Less Is Moor           7.5
Wave Pack              8.0
Weight of the World    8.0
Instrumental Relics    8.0
Jesus Is Born          7.4
Shrines                8.0
Colores                7.3
Name: Score, dtype: float64

In [80]:
from sklearn.metrics import mean_squared_error

np.sqrt(mean_squared_error(y_pred, y_test))

0.7266898696601376

In [81]:
y_pred-y_test

38 Baby 2             -1.414
BLQLYTE               -0.178
As God Intended       -0.640
Self Hate Wraith       0.193
High Off Life         -0.077
Less Is Moor          -0.295
Wave Pack             -0.808
Weight of the World   -0.996
Instrumental Relics   -1.073
Jesus Is Born         -0.317
Shrines               -0.926
Colores               -0.109
Name: Score, dtype: float64

In [75]:
mean_squared_error(y_train, forest_clf.predict(X_train))

0.16183187500000026